## Preparação dos dados

In [30]:
import numpy as np
data1 = np.load('datasets/Gabi.npy')
data2 = np.load('datasets/Jesse.npy')

data1.shape, data2.shape

((28, 1000, 4), (28, 1000, 4))

In [31]:
# Correção do número de eletrodos e unificação

# Como dito anteriormente, os dois primeiros eletrodos são funcionais
data1 = data1[:,:,:2]
data2 = data2[:,:,:2]

print(data1.shape, data2.shape)

data = np.array([data1, data2])
print(data.shape)

# Colocando os dados na dimensão final
data = data.transpose(0, 1, 3, 2)
print(data.shape)

(28, 1000, 2) (28, 1000, 2)
(2, 28, 1000, 2)
(2, 28, 2, 1000)


In [32]:
# Funções para aplicação dos filtros temporais
%matplotlib inline
from scipy import signal

def butter_bandpass(data, lowcut, highcut, fs=512, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)


def butter_notch(data, cutoff, var=1, fs=512, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

In [33]:
# Aplicação dos filtros
filtered_data = butter_notch(data, 60)
filtered_data = butter_bandpass(filtered_data, 5, 50)
filtered_data.shape

(2, 28, 2, 1000)

In [34]:

# Aplicação segmentação
data = filtered_data

step = 29
segment = 64
print('', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3, 4)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = signal.stft(data, fs=200, nperseg=segment, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes, ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes, ensaios, canais, janelas, linhas)')

 (2, 28, 2, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (2, 28, 33, 2, 64) - (classes, ensaios, canais, janelas, linhas)
Dominio da frequência:  (2, 28, 33, 2, 33) - (classes, ensaios, canais, janelas, linhas)


In [35]:
# Separar os participantes e salvar
print(chunks_freq[0].shape, chunks_freq[1].shape)
print(chunks_time[0].shape, chunks_time[1].shape)

p1_freq = chunks_freq[0]
p1_time = chunks_time[0]

p2_freq = chunks_freq[1]
p2_time = chunks_time[1]

np.save("datasets/p1_freq", p1_freq)
np.save("datasets/p1_time", p1_time)

np.save("datasets/p2_freq", p2_freq)
np.save("datasets/p2_time", p2_time)

(28, 33, 2, 33) (28, 33, 2, 33)
(28, 33, 2, 64) (28, 33, 2, 64)
